引用自：[使用NDArray来处理数据](http://zh.gluon.ai/chapter_crashcourse/ndarray.html)

#  使用NDArray来处理数据

对于机器学习来说，处理数据往往是万事之开头。它包含两个部分：
1. 数据读取，
2. 数据已经在内存中时如何处理。

本章将关注后者。

我们首先介绍 `NDArray`，这是 MXNet 储存和变换数据的主要工具。如果你之前用过`NumPy`，你会发现`NDArray`和`NumPy`的多维数组非常类似。当然，`NDArray`提供更多的功能，首先是CPU和GPU的异步计算，其次是自动求导。这两点使得`NDArray`能更好地支持机器学习。

##  让我们开始

我们先介绍最基本的功能。如果你不懂我们用到的数学操作也不用担心，例如按元素加法、正态分布；我们会在之后的章节分别从数学和代码编写的角度详细介绍。

我们首先从`mxnet`导入`ndarray`这个包

In [1]:
from mxnet import ndarray as nd
from mxnet import gluon
import mxnet as mx
import numpy as np

然后我们创建一个 $3$ 行和 $4$ 列的 2D 数组（通常也叫**矩阵**），并且把每个元素初始化成 $0$。

In [2]:
nd.zeros((3, 4))


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
<NDArray 3x4 @cpu(0)>

In [3]:
a = nd.zeros((3,4), ctx= mx.gpu(0))
a


[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
<NDArray 3x4 @gpu(0)>

In [4]:
a.dtype

numpy.float32

In [5]:
b = mx.nd.zeros((1,2), mx.cpu(), 'float16', stype='row_sparse')
b


<RowSparseNDArray 1x2 @cpu(0)>

In [6]:
b.stype

'row_sparse'

类似的，我们可以创建数组每个元素被初始化成 $1$。

In [7]:
x = nd.ones((3, 4))
x


[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
<NDArray 3x4 @cpu(0)>

或者从 python 的数组直接构造

In [8]:
nd.array([[1, 3], [4, 6]])


[[1. 3.]
 [4. 6.]]
<NDArray 2x2 @cpu(0)>

我们经常需要创建随机数组，即每个元素的值都是随机采样而来，这个经常被用来**初始化模型参数**。以下代码创建数组，它的元素服从均值 $0$ 标准差 $1$ 的正态分布。

In [9]:
y = nd.random_normal(0, 1, shape= (3, 4), ctx= mx.cpu(100))
y


[[ 1.1630785   0.4838046   0.29956347  0.15302546]
 [-1.1688148   1.5580711  -0.5459446  -2.3556297 ]
 [ 0.5414402   2.6785066   1.2546345  -0.54877394]]
<NDArray 3x4 @cpu(100)>

In [10]:
 %timeit nd.random_normal(shape= (3, 400, 5), ctx= mx.gpu())

41.6 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [11]:
 %timeit nd.random_normal(shape= (3, 400, 5), ctx= mx.cpu())

45.8 µs ± 632 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


跟 `NumPy` 一样，每个数组的形状可以通过 `.shape` 来获取：

In [12]:
y.shape

(3, 4)

它的大小，就是总元素个数，是形状的累乘

In [13]:
y.size

12

## 操作符

NDArray 支持大量的数学操作符：

### 按元素计算

In [14]:
x + y            # 加法


[[ 2.1630785   1.4838046   1.2995634   1.1530255 ]
 [-0.16881478  2.5580711   0.45405543 -1.3556297 ]
 [ 1.5414402   3.6785066   2.2546344   0.45122606]]
<NDArray 3x4 @cpu(0)>

In [15]:
x * y        # 乘法


[[ 1.1630785   0.4838046   0.29956347  0.15302546]
 [-1.1688148   1.5580711  -0.5459446  -2.3556297 ]
 [ 0.5414402   2.6785066   1.2546345  -0.54877394]]
<NDArray 3x4 @cpu(0)>

In [16]:
nd.exp(y)    # 指数运算


[[ 3.1997688   1.6222347   1.3492696   1.1653546 ]
 [ 0.31073502  4.749651    0.5792943   0.09483377]
 [ 1.71848    14.563329    3.5065565   0.57765764]]
<NDArray 3x4 @cpu(100)>

也可以转置一个矩阵然后计算矩阵乘法，即：

### 矩阵运算

In [17]:
nd.dot(x, y.T)


[[ 2.099472  -2.512318   3.9258072]
 [ 2.099472  -2.512318   3.9258072]
 [ 2.099472  -2.512318   3.9258072]]
<NDArray 3x3 @cpu(0)>

##  广播（Broadcasting）

当二元操作符左右两边 ndarray 形状不一样时，系统会尝试将其复制到一个共同的形状。例如 `a` 的第 $0$ 维是 $3$, `b` 的第 $0$ 维是 $1$，那么`a+b` 时会将 `b` 沿着第 $0$ 维复制 $3$ 遍：

In [18]:
a = nd.arange(3).reshape((3,1))
b = nd.arange(2).reshape((1,2))

In [19]:
a


[[0.]
 [1.]
 [2.]]
<NDArray 3x1 @cpu(0)>

In [20]:
b


[[0. 1.]]
<NDArray 1x2 @cpu(0)>

In [21]:
a + b


[[0. 1.]
 [1. 2.]
 [2. 3.]]
<NDArray 3x2 @cpu(0)>

## 跟 NumPy 的转换

ndarray 可以很方便地同 numpy 进行转换

In [22]:
import numpy as np
x = np.ones((2,3))
y = nd.array(x)  # numpy -> mxnet
z = y.asnumpy()  # mxnet -> numpy
print([z, y])

[array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32), 
[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>]


## 替换操作

在前面的样例中，我们为每个操作新开内存来存储它的结果。例如，如果我们写 `y = x + y`, 我们会把 `y` 从现在指向的实例转到新建的实例上去。我们可以用 Python 的`id()`函数来看这个是怎么执行的：

In [22]:
x = nd.ones((3, 4))
y = nd.ones((3, 4))

before = id(y)
y = y + x
id(y) == before

False

我们可以把结果通过 `[:]` 写到一个之前开好的数组里：

In [23]:
z = nd.zeros_like(x)
z


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
<NDArray 3x4 @cpu(0)>

In [24]:
before = id(z)
z[:] = x + y
id(z) == before

True

In [25]:
z


[[3. 3. 3. 3.]
 [3. 3. 3. 3.]
 [3. 3. 3. 3.]]
<NDArray 3x4 @cpu(0)>

In [26]:
z[:] = 7
z


[[7. 7. 7. 7.]
 [7. 7. 7. 7.]
 [7. 7. 7. 7.]]
<NDArray 3x4 @cpu(0)>

In [27]:
id(z) == before

True

观上述信息，可以知道 `[:]` 操作只将原对象就地修改，即仅仅修改对象的标签（标识符）。

我们还可以为 `x+y` 创建临时空间，然后再复制到 `z`。为了避免浪费内存开销，我们可以使用操作符的全名版本中的 `out` 参数：

In [28]:
nd.elemwise_add(x, y, out= z)


[[3. 3. 3. 3.]
 [3. 3. 3. 3.]
 [3. 3. 3. 3.]]
<NDArray 3x4 @cpu(0)>

In [29]:
id(z) == before

True

如果现有的数组不会复用，我们也可以用 `x[:] = x + y` ，或者 `x += y` 达到这个目的：

In [30]:
before = id(x)
x += y
id(x) == before

True

## 1.7 截取（Slicing）

NXNet NDArray 提供了各种截取方法。截取 x 的 index 为 $1, 2$ 的列：

In [31]:
x = nd.arange(0,9).reshape((3,3))
print('x: ', x)
x[1:3]

x:  
[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>



[[3. 4. 5.]
 [6. 7. 8.]]
<NDArray 2x3 @cpu(0)>

以及直接写入指定位置：

In [32]:
x[1,2] = 9.0
x


[[0. 1. 2.]
 [3. 4. 9.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>

多维截取:

In [33]:
x = nd.arange(0,9).reshape((3,3))
print('x: ', x)
x[1:2, 1:3]

x:  
[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>



[[4. 5.]]
<NDArray 1x2 @cpu(0)>

多维写入：

In [34]:
x[1:2, 1:3] = 9.0
x


[[0. 1. 2.]
 [3. 9. 9.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>

## 总结

ndarray 模块提供一系列多维数组操作函数。所有函数列表可以参见[NDArray API文档](https://mxnet.incubator.apache.org/api/python/ndarray.html)。

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/745)

## 小灶

In [27]:
# define a nd.array like np.array
a = mx.nd.random_exponential(shape=(2, 3))   # 指数分布
b = mx.nd.ones((3, 4), dtype= np.int32)
c = mx.nd.full((2, 3), 3) # get a 2x3 matrix with every item 3
d = mx.nd.array([[2, 3], [5, 9]], ctx=mx.gpu(0))

In [28]:
a


[[0.98948145 0.24877903 0.3050495 ]
 [0.00561476 2.7101123  2.216487  ]]
<NDArray 2x3 @cpu(0)>

In [29]:
c


[[3. 3. 3.]
 [3. 3. 3.]]
<NDArray 2x3 @cpu(0)>

In [30]:
d


[[2. 3.]
 [5. 9.]]
<NDArray 2x2 @gpu(0)>

In [31]:
d.context

gpu(0)

In [32]:
# show it implicity
b.asnumpy()

array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]])

In [33]:
b.context

cpu(0)

In [44]:
a = mx.nd.ones((100,100), mx.cpu())
b = mx.nd.ones((100,100), mx.gpu()) + 2
c = mx.nd.ones((100,100), mx.gpu()) + 5
a.copyto(c)  # copy from CPU to GPU
d = a.as_in_context(c.context)  # same to above

In [45]:
a + b.as_in_context(a.context)


[[4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]
 ...
 [4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]]
<NDArray 100x100 @cpu(0)>

In [53]:
x = mx.nd.ones((2,3))
y = mx.nd.zeros((2,3), mx.gpu(0)) + 2
z = x.copyto(y)
z is y

True

In [54]:
x


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>

In [55]:
y


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @gpu(0)>

In [49]:
y.copyto(mx.gpu(0))


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @gpu(0)>